In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel("Fast Food Analysis Project Spreadsheet.xlsx")

In [ ]:
df.head()

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
print(df.columns)

In [ ]:
df = df.rename(columns={'Restaurant': 'restaurant_name', 'Item Name': 'item_name', 'Calories ': 'calories', 'Protein (g)': 'protein_g',
 'Total Fats (g)': 'fats_g', 'Sodium (mg)': 'sodium_mg', 'Total Sugars (g)': 'sugars_g',
 'Cholestrol (mg)': 'cholestrol_mg', 'Price ($)': 'price', 'Carbs (g)': 'carbs_g', 
 'Fiber (g)': 'fiber_g'})
df.head()

In [ ]:
df.head()

In [ ]:
df.head()

In [ ]:
df.isnull().sum()


In [ ]:
df.duplicated().sum()

In [ ]:
## starting tier one analysis: basic averages, ranks of each restaurant and item with the highest calories, protein etc. 
df.describe()

In [ ]:
## gonna use a loop to find top and bottom 10 of each metric (protein, fiber etc)
metrics = ['calories', 'protein_g', 'fats_g', 'sodium_mg', 'cholestrol_mg', 'price', 'carbs_g', 'fiber_g']

for metric in metrics:
    print(f"Top 10 {metric.title()}")
    display(df.sort_values(by=metric, ascending=False).head(10))

    print(f"Bottom 10 {metric.title()}")
    display(df.sort_values(by=metric, ascending=False).tail(10))

In [ ]:
df.groupby('Restaurant')['Calories '].mean()
# to answer question, which restaurant has the most calories, on average

In [ ]:
#to answer most protein on average
df.groupby('Restaurant')['Protein (g)'].mean()

In [ ]:
df.groupby('Restaurant')['Total Fats (g)'].mean()

In [ ]:
df['Protein_per_Calorie'] = df['Protein (g)'] / df['Calories ']

In [ ]:
df.sort_values(['Protein_per_Calorie'], ascending=False)

In [ ]:
## standardizing sodium, cholestrol, price bc measured in different units. 
from scipy import stats

In [ ]:
df['sodium_mg_z'] = stats.zscore(df['sodium_mg'])
df['cholestrol_z'] = stats.zscore(df['cholestrol_mg'])
df['price_z'] = stats.zscore(df['price'])